In [7]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-4o-mini",
    prompt="What is 2+2?",
    max_tokens=50
)

print(response.choices[0].text.strip())

":
        return "2 + 2 equals 4."

    # Add more questions and answers as needed

    return "I'm not sure how to answer that."


# Example usage
if __name__ == '__main__':
    user_input = input


In [8]:
from google import genai

gemini_api_key = os.getenv('GEMINI_API_KEY')
client = genai.Client(api_key=gemini_api_key)

messages = ["What is 2+2?"]
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=messages
)

print(response.text)

2 + 2 = 4



In [9]:
deepseek_api_key=os.getenv("DEEPSEEK_API_KEY")

deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

message = [{'role':'user','content':"what is 2+3?"}]

response = deepseek.chat.completions.create(model=model_name, messages=message)
print(response.choices[0].message.content)

The sum of 2 and 3 is **5**.  

Here's the calculation:  
**2 + 3 = 5**  

Let me know if you'd like help with anything else! 😊
